# Preprocessing

Installing, Importing and Preprocessing

In [997]:
# Install and import spacy and plotly.
%pip install spaCy
%pip install plotly
%pip install nbformat --upgrade
%pip install pyspellchecker
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.7 MB/s eta 0:00:00:00:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [998]:
# Import spacy
import spacy

# Import os to upload documents and metadata
import os

# Import pandas DataFrame packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


#Import typo and marks correction packages
from spellchecker import SpellChecker





In [999]:
# Create empty lists for file names and contents
texts = []
file_names = []

# Iterate through each file in the folder
for _file_name in os.listdir('txt_files'):
# Look for only text files
    if _file_name.endswith('.txt'):
    # Append contents of each text file to text list
        texts.append(open('txt_files' + '/' + _file_name, 'r', encoding='utf-8').read())
        # Append name of each file to file name list
        file_names.append(_file_name)

In [1000]:
# Create dictionary object associating each file name with its text
d = {'Filename':file_names,'Text':texts}

In [1001]:
# Turn dictionary into a dataframe
lyric_df = pd.DataFrame(d)

In [1002]:
lyric_df.head()


,Filename,Text
0,Flowers In December.txt,Before I let you down again\nI just want to se...
1,Roseblood.txt,Everyday you can see\nChanges in her hair and ...
2,I'm Sailin'.txt,"I'm sailin', sailin'\n(To a) place I've never ..."
3,Still Cold.txt,You really had a million hearts to break\nAll ...
4,"Quiet, The Winter Harbor.txt",Well you're still walking\nAround the block\nY...


The beginnings of some of the texts may contain extra spaces (indicated by \t or \n). These characters can be replaced by a single space using the str.replace() method.

In [1003]:
lyric_df['Document'] = lyric_df['Text'].copy()

# Remove extra spaces from papers
lyric_df['Text'] = lyric_df['Text'].str.replace('\s+', ' ', regex=True).str.strip()
lyric_df.head()


,Filename,Text,Document
0,Flowers In December.txt,Before I let you down again I just want to see...,Before I let you down again\nI just want to se...
1,Roseblood.txt,Everyday you can see Changes in her hair and s...,Everyday you can see\nChanges in her hair and ...
2,I'm Sailin'.txt,"I'm sailin', sailin' (To a) place I've never s...","I'm sailin', sailin'\n(To a) place I've never ..."
3,Still Cold.txt,You really had a million hearts to break All t...,You really had a million hearts to break\nAll ...
4,"Quiet, The Winter Harbor.txt",Well you're still walking Around the block You...,Well you're still walking\nAround the block\nY...


checking typos by using spellchecker

In [1004]:

spell = SpellChecker()

# Create a copy of lyric_df
typo_df = lyric_df.copy()

def find_typos(text):
    misspelled_words = spell.unknown(text.split())
    return list(misspelled_words)

# Apply the function to each row in the 'Text' column
typo_df['Typos'] = typo_df['Text'].apply(find_typos)

# Display the DataFrame with the identified typos
typo_df


,Filename,Text,Document,Typos
0,Flowers In December.txt,Before I let you down again I just want to see...,Before I let you down again\nI just want to se...,"[december, i've]"
1,Roseblood.txt,Everyday you can see Changes in her hair and s...,Everyday you can see\nChanges in her hair and ...,[ok]
2,I'm Sailin'.txt,"I'm sailin', sailin' (To a) place I've never s...","I'm sailin', sailin'\n(To a) place I've never ...","[orleans, rainin', sam, (to, blowin', i'm, i'v..."
3,Still Cold.txt,You really had a million hearts to break All t...,You really had a million hearts to break\nAll ...,"[places,]"
4,"Quiet, The Winter Harbor.txt",Well you're still walking Around the block You...,Well you're still walking\nAround the block\nY...,"[say,, i'd, i'm, ""come, over"", well,]"
5,Five String Serenade.txt,This is my five string serenade Beneath the wa...,This is my five string serenade\nBeneath the w...,[i'm]
6,That Way Again.txt,He says that he tries to solve all his problem...,He says that he tries to solve all his problem...,"[on,, i'm]"
7,Flying Low.txt,"Well, I'm flying low in London's past days It'...","Well, I'm flying low in London's past days\nIt...","[he,, you,, who,, london's, i'm, parlour, well,]"
8,Lay Myself Down.txt,"Just like that, she follows me My wings are un...","Just like that, she follows me\nMy wings are u...","[that,, i'll, far,]"
9,Fade Into You.txt,I want to hold the hand inside you I want to t...,I want to hold the hand inside you\nI want to ...,"[life,]"


We  need to fix document by correcting text column.Firstly, replacing "in'" by"ing".

In [1005]:

lyric_df_copy = lyric_df.copy()
# Define a regular expression pattern to match consonant letters followed by 'in'
pattern= r'([obcdfghjklmqnprstvwxyz])in\''



# Use str.replace() with the regular expression to replace the pattern with 'ing'
lyric_df_copy['Filename'] = lyric_df_copy['Filename'].str.replace(pattern, r'\1ing', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace(pattern, r'\1ing', regex=True)

# Print the DataFrame after replacements
lyric_df_copy.head()


,Filename,Text,Document
0,Flowers In December.txt,Before I let you down again I just want to see...,Before I let you down again\nI just want to se...
1,Roseblood.txt,Everyday you can see Changes in her hair and s...,Everyday you can see\nChanges in her hair and ...
2,I'm Sailing.txt,"I'm sailing, sailing (To a) place I've never s...","I'm sailin', sailin'\n(To a) place I've never ..."
3,Still Cold.txt,You really had a million hearts to break All t...,You really had a million hearts to break\nAll ...
4,"Quiet, The Winter Harbor.txt",Well you're still walking Around the block You...,Well you're still walking\nAround the block\nY...


Then,remove all the marks that are mistakes.

In [1006]:


lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace(r'\(|\)', '', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace(',', ' ', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace('\.\.\.', ' ', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace('\?', ' ', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace('�', ' ', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace('"', ' ', regex=True)
lyric_df_copy['Text'] = lyric_df_copy['Text'].str.replace('.', ' ', regex=True)





check typo again by using Spellchecker

In [1007]:
spell = SpellChecker()

def find_typos(text):
    misspelled_words = spell.unknown(text.split())
    return list(misspelled_words)


lyric_df_copy['Typos'] = lyric_df_copy['Text'].apply(find_typos)

lyric_df_copy[['Filename','Text', 'Typos']]

,Filename,Text,Typos
0,Flowers In December.txt,Before I let you down again I just want to see...,"[december, i've]"
1,Roseblood.txt,Everyday you can see Changes in her hair and s...,[ok]
2,I'm Sailing.txt,I'm sailing sailing To a place I've never see...,"[orleans, sam, i'm, i've]"
3,Still Cold.txt,You really had a million hearts to break All t...,[]
4,"Quiet, The Winter Harbor.txt",Well you're still walking Around the block You...,"[i'd, i'm]"
5,Five String Serenade.txt,This is my five string serenade Beneath the wa...,[i'm]
6,That Way Again.txt,He says that he tries to solve all his problem...,[i'm]
7,Flying Low.txt,Well I'm flying low in London's past days It'...,"[london's, i'm, parlour]"
8,Lay Myself Down.txt,Just like that she follows me My wings are un...,[i'll]
9,Fade Into You.txt,I want to hold the hand inside you I want to t...,[]


It seems alrigh. Then replacing the texts with errors by the corrected texts.

In [1008]:
lyric_df['Text']=lyric_df_copy['Text']
lyric_df.head()

,Filename,Text,Document
0,Flowers In December.txt,Before I let you down again I just want to see...,Before I let you down again\nI just want to se...
1,Roseblood.txt,Everyday you can see Changes in her hair and s...,Everyday you can see\nChanges in her hair and ...
2,I'm Sailin'.txt,I'm sailing sailing To a place I've never see...,"I'm sailin', sailin'\n(To a) place I've never ..."
3,Still Cold.txt,You really had a million hearts to break All t...,You really had a million hearts to break\nAll ...
4,"Quiet, The Winter Harbor.txt",Well you're still walking Around the block You...,Well you're still walking\nAround the block\nY...


Reading metadata CSV file

In [1009]:
# Load metadata.
metadata_df = pd.read_csv('metadata.csv')
metadata_df.head()


,TITLE,ALBUM,YEAR,TYPE
0,Be My Angel,She Hangs Brightly,1990,ALBUM
1,Before I Sleep,She Hangs Brightly,1990,ALBUM
2,Blue Flower,She Hangs Brightly,1990,ALBUM
3,Free,She Hangs Brightly,1990,ALBUM
4,Ghost Highway,She Hangs Brightly,1990,ALBUM


In [1010]:
# Remove .txt from title of each paper
lyric_df['Filename'] = lyric_df['Filename'].str.replace('.txt', '', regex=True)

# Rename column from paper ID to Title
metadata_df.rename(columns={"TITLE": "Filename"}, inplace=True)

In [1011]:
# Merge metadata and papers into new DataFrame
# Will only keep rows where both essay and metadata are present
final_lyric_df = metadata_df.merge(lyric_df,on='Filename')
final_lyric_df = final_lyric_df.rename(columns={'Filename': 'TITLE'})

Let's check the head of the DataFrame again to confirm everything has worked well. Check the first five rows to make sure each has a title, album, year, type and text (the full lyric)

In [1012]:
# Print DataFrame
final_lyric_df.head()


,TITLE,ALBUM,YEAR,TYPE,Text,Document
0,Be My Angel,She Hangs Brightly,1990,ALBUM,They say it's me that makes you do things you ...,They say it's me\nthat makes you do things\nyo...
1,Before I Sleep,She Hangs Brightly,1990,ALBUM,If it's The truth That's all I know I look for...,If it's\nThe truth\nThat's all I know\nI look ...
2,Blue Flower,She Hangs Brightly,1990,ALBUM,Waiting for a sign from you Waiting for a sign...,Waitin' for a sign from you\nWaitin' for a sig...
3,Free,She Hangs Brightly,1990,ALBUM,I fell asleep in the silence Before the street...,I fell asleep in the silence\nBefore the stree...
4,Ghost Highway,She Hangs Brightly,1990,ALBUM,You're a ghost on the highway And I'll love yo...,You're a ghost on the highway\nAnd I'll love y...


The resulting DataFrame is now ready for analysis.


## Text Enrichment with spaCy

### Creating Doc Objects

To use spaCy, the first step is to load one of spaCy’s Trained Models and Pipelines which will be used to perform tokenization, part-of-speech tagging, and other text enrichment tasks. A wide range of options are available (see the full list here), and they vary based on size and language.

We’ll use en_core_web_sm, which has been trained on written web texts. It may not perform as accurately as the those trained on medium and large English language models, but it will deliver results most efficiently. Once we’ve loaded en_core_web_sm, we can check what actions it performs; parser, tagger, lemmatizer, and NER, should be among those listed.

In [1013]:
# Load nlp pipeline
nlp = spacy.load('en_core_web_sm')

# Check what functions it performs
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [1014]:
#Define example sentence
sentence = "This is 'an' example? sentence"

# Call the nlp model on the sentence
doc = nlp(sentence)

Next we can call on the Doc object to get the information we’re interested in. The command below loops through each token in a Doc object and prints each word in the text along with its corresponding part-of-speech:

In [1015]:
# Loop through each token in doc object
for token in doc:
    # Print text and part of speech for each
    print(token.text, token.pos_)

This PRON
is AUX
' PUNCT
an DET
' PUNCT
example NOUN
? PUNCT
sentence NOUN


Let’s try the same process on the lyrics . As we’ll be calling the NLP function on every text in the DataFrame, we should first define a function that runs nlp on whatever input text is given. Functions are a useful way to store operations that will be run multiple times, reducing duplications and improving code readability.

In [1016]:
# Define a function that runs the nlp pipeline on any given input text
def process_text(text):
    return nlp(text)

After the function is defined, use .apply() to apply it to every cell in a given DataFrame column. In this case, nlp will run on each cell in the Text column of the final_lyric_df DataFrame, creating a Doc object from every lyric text. These Doc objects will be stored in a new column of the DataFrame called Doc.

Running this function takes several minutes because spaCy is performing all the parsing and tagging tasks on each text. However, when it is complete, we can simply call on the resulting Doc objects to get parts-of-speech, named entities, and other information of interest, just as in the example of the sentence above.

In [1017]:
# Apply the function to the "Text" column, so that the nlp pipeline is called on each student lyric
final_lyric_df['Doc'] = final_lyric_df['Text'].apply(process_text)

## Text Reduction

### Tokenization

A critical first step spaCy performs is tokenization, or the segmentation of strings into individual words and punctuation markers. Tokenization enables spaCy to parse the grammatical structures of a text and identify characteristics of each word-like part-of-speech.

To retrieve a tokenized version of each text in the DataFrame, we’ll write a function that iterates through any given Doc object and returns all functions found within it.

In [1018]:
# Define a function to retrieve tokens from a doc object
def get_token(doc):
    return [(token.text) for token in doc]

As with the function used to create Doc objects, the token function can be applied to the DataFrame. In this case, we will call the function on the Doc column, since this is the column which stores the results from the processing done by spaCy.

In [1019]:
# Run the token retrieval function on the doc objects in the dataframe
final_lyric_df['Tokens'] = final_lyric_df['Doc'].apply(get_token)
final_lyric_df.head()

,TITLE,ALBUM,YEAR,TYPE,Text,Document,Doc,Tokens
0,Be My Angel,She Hangs Brightly,1990,ALBUM,They say it's me that makes you do things you ...,They say it's me\nthat makes you do things\nyo...,"(They, say, it, 's, me, that, makes, you, do, ...","[They, say, it, 's, me, that, makes, you, do, ..."
1,Before I Sleep,She Hangs Brightly,1990,ALBUM,If it's The truth That's all I know I look for...,If it's\nThe truth\nThat's all I know\nI look ...,"(If, it, 's, The, truth, That, 's, all, I, kno...","[If, it, 's, The, truth, That, 's, all, I, kno..."
2,Blue Flower,She Hangs Brightly,1990,ALBUM,Waiting for a sign from you Waiting for a sign...,Waitin' for a sign from you\nWaitin' for a sig...,"(Waiting, for, a, sign, from, you, Waiting, fo...","[Waiting, for, a, sign, from, you, Waiting, fo..."
3,Free,She Hangs Brightly,1990,ALBUM,I fell asleep in the silence Before the street...,I fell asleep in the silence\nBefore the stree...,"(I, fell, asleep, in, the, silence, Before, th...","[I, fell, asleep, in, the, silence, Before, th..."
4,Ghost Highway,She Hangs Brightly,1990,ALBUM,You're a ghost on the highway And I'll love yo...,You're a ghost on the highway\nAnd I'll love y...,"(You, 're, a, ghost, on, the, highway, And, I,...","[You, 're, a, ghost, on, the, highway, And, I,..."


If we compare the Text and Tokens column, we find a couple of differences. In the table below, you’ll notice that most importantly, the words, spaces, and punctuation markers in the Tokens column are separated by commas, indicating that each have been parsed as individual tokens. The text in the Tokens column is also bracketed; this indicates that tokens have been generated as a list.

In [1020]:
tokens = final_lyric_df[['Text', 'Tokens']].copy()
tokens.head()



,Text,Tokens
0,They say it's me that makes you do things you ...,"[They, say, it, 's, me, that, makes, you, do, ..."
1,If it's The truth That's all I know I look for...,"[If, it, 's, The, truth, That, 's, all, I, kno..."
2,Waiting for a sign from you Waiting for a sign...,"[Waiting, for, a, sign, from, you, Waiting, fo..."
3,I fell asleep in the silence Before the street...,"[I, fell, asleep, in, the, silence, Before, th..."
4,You're a ghost on the highway And I'll love yo...,"[You, 're, a, ghost, on, the, highway, And, I,..."


### Lemmatization

Another process performed by spaCy is lemmatization, or the retrieval of the dictionary root word of each word (for example “brighten” for “brightening”). We’ll perform a similar set of steps to those above to create a function to call the lemmas from the Doc object, then apply it to the DataFrame.

In [1021]:
# Define a function to retrieve lemmas from a doc object
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

# Run the lemma retrieval function on the doc objects in the dataframe
final_lyric_df['Lemmas'] = final_lyric_df['Doc'].apply(get_lemma)

Lemmatization can help reduce noise and refine results for researchers who are conducting keyword searches. For example, let’s compare counts of the word “she” in the original Tokens column and in the lemmatized Lemmas column.

In [1022]:
print(f'"she" appears in the text tokens column ' + str(final_lyric_df['Tokens'].apply(lambda x: x.count('she')).sum()) + ' times.')
print(f'"she" appears in the lemmas column ' + str(final_lyric_df['Lemmas'].apply(lambda x: x.count('she')).sum()) + ' times.')

"she" appears in the text tokens column 19 times.
"she" appears in the lemmas column 50 times.


As expected, there are more instances of “she” in the Lemmas column, as the lemmatization process has grouped inflected word forms (her,hers) into the base word “she.”

## Text Annotation

### Part of Speech Tagging

spaCy facilitates two levels of part-of-speech tagging: coarse-grained tagging, which predicts the simple universal part-of-speech of each token in a text (such as noun, verb, adjective, adverb), and detailed tagging, which uses a larger, more fine-grained set of part-of-speech tags (for example 3rd person singular present verb). The part-of-speech tags used are determined by the English language model we use. In this case, we’re using the small English model, and you can explore the differences between the models on spaCy’s website.

We can call the part-of-speech tags in the same way as the lemmas. Create a function to extract them from any given Doc object and apply the function to each Doc object in the DataFrame. The function we’ll create will extract both the coarse- and fine-grained part-of-speech for each token (token.pos_ and token.tag_, respectively).

In [1023]:
# Define a function to retrieve lemmas from a doc object
def get_pos(doc):
    #Return the coarse- and fine-grained part of speech text for each token in the doc
    return [(token.pos_, token.tag_) for token in doc]

# Define a function to retrieve parts of speech from a doc object
final_lyric_df['POS'] = final_lyric_df['Doc'].apply(get_pos)


We can create a list of the part-of-speech columns to review them further. The first (coarse-grained) tag corresponds to a generally recognizable part-of-speech such as a noun, adjective, or punctuation mark, while the second (fine-grained) category are a bit more difficult to decipher.

In [1024]:
# Create a list of part of speech tags
list(final_lyric_df['POS'])

final_lyric_df['POS']


0     [(PRON, PRP), (VERB, VBP), (PRON, PRP), (AUX, ...
1     [(SCONJ, IN), (PRON, PRP), (AUX, VBZ), (DET, D...
2     [(VERB, VBG), (ADP, IN), (DET, DT), (NOUN, NN)...
3     [(PRON, PRP), (VERB, VBD), (ADJ, JJ), (ADP, IN...
4     [(PRON, PRP), (AUX, VBP), (DET, DT), (NOUN, NN...
5     [(INTJ, UH), (PRON, PRP), (VERB, VBP), (PRON, ...
6     [(VERB, VB), (PRON, PRP), (ADP, IN), (NOUN, NN...
7     [(PRON, PRP), (VERB, VBZ), (ADV, RB), (ADP, IN...
8     [(ADJ, JJ), (INTJ, UH), (ADJ, JJ), (DET, DT), ...
9     [(NOUN, NNS), (VERB, VBP), (ADP, IN), (DET, DT...
10    [(PRON, EX), (VERB, VBZ), (DET, DT), (ADJ, JJ)...
11    [(PRON, PRP), (VERB, VBP), (PART, TO), (VERB, ...
12    [(PRON, DT), (AUX, VBZ), (PRON, PRP$), (NUM, C...
13    [(ADV, RB), (VERB, VBG), (PROPN, NNP), (CCONJ,...
14    [(INTJ, UH), (PROPN, NNP), (ADP, IN), (NOUN, N...
15    [(PRON, PRP), (AUX, VBZ), (PRON, PRP$), (NOUN,...
16    [(VERB, VB), (ADP, RP), (NOUN, NN), (ADP, IN),...
17    [(VERB, VBD), (PRON, PRP), (VERB, VBG), (A

Fortunately, spaCy has a built-in function called explain that can provide a short description of any tag of interest. If we try it on the tag IN using spacy.explain("IN"), the output reads conjunction, subordinating or preposition.

In [1025]:
spacy.explain("IN")

'conjunction, subordinating or preposition'

In some cases, you may want to get only a set of part-of-speech tags for further analysis, like all of the proper nouns. A function can be written to perform this task, extracting only words which have been fitted with the proper noun tag.

In [1026]:
# Define function to extract proper nouns from Doc object
def extract_proper_nouns(doc):
    return [token.text for token in doc if token.pos_ == 'PROPN']

# Apply function to Doc column and store resulting proper nouns in new column
final_lyric_df['Proper_Nouns'] = final_lyric_df['Doc'].apply(extract_proper_nouns)

#### Listing the nouns in each text can help us ascertain the texts’ subjects. Let’s list the nouns in two different texts, the text located in row 5 of the DataFrame and the text located in row 40.

In [1027]:
list(final_lyric_df.loc[[5, 40], 'Proper_Nouns'])



[['Baby', 'Baby', 'Baby', 'Baby'], ['God', 'Palm', 'Shore', 'God']]

The first text in the list includes botany and astronomy concepts; this is likely to have been written for a biology course. In contrast, the second text appears to be an analysis of Shakespeare plays and movie adaptations, likely written for an English course.

### Named Entity Recognition

spaCy can tag named entities in the text, such as names, dates, organizations, and locations. Call the full list of named entities and their descriptions using this code:

In [1028]:
# Get all NE labels and assign to variable
labels = nlp.get_pipe("ner").labels

# Print each label and its description
for label in labels:
    print(label + ' : ' + spacy.explain(label))

CARDINAL : Numerals that do not fall under another type
DATE : Absolute or relative dates or periods
EVENT : Named hurricanes, battles, wars, sports events, etc.
FAC : Buildings, airports, highways, bridges, etc.
GPE : Countries, cities, states
LANGUAGE : Any named language
LAW : Named documents made into laws.
LOC : Non-GPE locations, mountain ranges, bodies of water
MONEY : Monetary values, including unit
NORP : Nationalities or religious or political groups
ORDINAL : "first", "second", etc.
ORG : Companies, agencies, institutions, etc.
PERCENT : Percentage, including "%"
PERSON : People, including fictional
PRODUCT : Objects, vehicles, foods, etc. (not services)
QUANTITY : Measurements, as of weight or distance
TIME : Times smaller than a day
WORK_OF_ART : Titles of books, songs, etc.


We’ll create a function to extract the named entity tags from each Doc object and apply it to the Doc objects in the DataFrame, storing the named entities in a new column:

In [1029]:
# Define function to extract named entities from doc objects
def extract_named_entities(doc):
    return [ent.label_ for ent in doc.ents]

# Apply function to Doc column and store resulting named entities in new column
final_lyric_df['Named_Entities'] = final_lyric_df['Doc'].apply(extract_named_entities)
final_lyric_df['Named_Entities']

0                                          [TIME, DATE]
1                                                    []
2                      [TIME, ORG, LANGUAGE, TIME, ORG]
3                                         [PERSON, ORG]
4                                      [PERSON, PERSON]
5                                                    []
6                                                    []
7                                              [PERSON]
8                                                    []
9                                                [TIME]
10                                     [PERSON, PERSON]
11                                                   []
12                                 [CARDINAL, CARDINAL]
13       [PERSON, CARDINAL, CARDINAL, PERSON, CARDINAL]
14    [PERSON, PERSON, PERSON, ORDINAL, PERSON, PERS...
15                       [DATE, DATE, DATE, DATE, DATE]
16                    [ORG, TIME, PERSON, PERSON, DATE]
17                                              

We can add another column with the words and phrases identified as named entities:

In [1030]:
# Define function to extract text tagged with named entities from doc objects
def extract_named_entities(doc):
    return [ent for ent in doc.ents]

# Apply function to Doc column and store resulting text in new column
final_lyric_df['NE_Words'] = final_lyric_df['Doc'].apply(extract_named_entities)
final_lyric_df['NE_Words']


0                            [(the, night), (the, day)]
1                                                    []
2     [(morning), (Superstar), (Flower), (morning), ...
3                                    [(Afraid), (Plea)]
4                                    [(Ghost), (Ghost)]
5                                                    []
6                                                    []
7                                              [(Sour)]
8                                                    []
9                                  [(the, night, Like)]
10                        [(Shine, There, 's), (Miles)]
11                                                   []
12                                     [(five), (five)]
13    [(Breathless), (two), (two), (Breathless), (two)]
14    [(Mary, Come), (Mary), (Mary), (first), (Mary)...
15    [(Yesterday), (today), (Yesterday), (another, ...
16    [(Fells), (tonight), (Sunshine), (Sunshine), (...
17                                              

Let’s visualize the words and their named entity tags in a single text. Call the first text’s Doc object and use displacy.render to visualize the text with the named entities highlighted and tagged:

In [1031]:
# Extract the first Doc object
doc = final_lyric_df['Doc'][40]

# Visualize named entity tagging in a single paper
displacy.render(doc, style='ent', jupyter=True)

/Users/cassielu/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


# Download Enriched Dataset

To save the dataset of doc objects, text reductions and linguistic annotations generated with spaCy, download the final_lyric_df DataFrame to your local computer as a .csv file:

In [1032]:
final_lyric_df.head()

,TITLE,ALBUM,YEAR,TYPE,Text,Document,Doc,Tokens,Lemmas,POS,Proper_Nouns,Named_Entities,NE_Words
0,Be My Angel,She Hangs Brightly,1990,ALBUM,They say it's me that makes you do things you ...,They say it's me\nthat makes you do things\nyo...,"(They, say, it, 's, me, that, makes, you, do, ...","[They, say, it, 's, me, that, makes, you, do, ...","[they, say, it, be, I, that, make, you, do, th...","[(PRON, PRP), (VERB, VBP), (PRON, PRP), (AUX, ...",[],"[TIME, DATE]","[(the, night), (the, day)]"
1,Before I Sleep,She Hangs Brightly,1990,ALBUM,If it's The truth That's all I know I look for...,If it's\nThe truth\nThat's all I know\nI look ...,"(If, it, 's, The, truth, That, 's, all, I, kno...","[If, it, 's, The, truth, That, 's, all, I, kno...","[if, it, be, the, truth, that, be, all, I, kno...","[(SCONJ, IN), (PRON, PRP), (AUX, VBZ), (DET, D...",[],[],[]
2,Blue Flower,She Hangs Brightly,1990,ALBUM,Waiting for a sign from you Waiting for a sign...,Waitin' for a sign from you\nWaitin' for a sig...,"(Waiting, for, a, sign, from, you, Waiting, fo...","[Waiting, for, a, sign, from, you, Waiting, fo...","[wait, for, a, sign, from, you, wait, for, a, ...","[(VERB, VBG), (ADP, IN), (DET, DT), (NOUN, NN)...","[Flower, Superstar, Flower, Superstar]","[TIME, ORG, LANGUAGE, TIME, ORG]","[(morning), (Superstar), (Flower), (morning), ..."
3,Free,She Hangs Brightly,1990,ALBUM,I fell asleep in the silence Before the street...,I fell asleep in the silence\nBefore the stree...,"(I, fell, asleep, in, the, silence, Before, th...","[I, fell, asleep, in, the, silence, Before, th...","[I, fall, asleep, in, the, silence, before, th...","[(PRON, PRP), (VERB, VBD), (ADJ, JJ), (ADP, IN...","[Lay, Afraid, Free, Plea]","[PERSON, ORG]","[(Afraid), (Plea)]"
4,Ghost Highway,She Hangs Brightly,1990,ALBUM,You're a ghost on the highway And I'll love yo...,You're a ghost on the highway\nAnd I'll love y...,"(You, 're, a, ghost, on, the, highway, And, I,...","[You, 're, a, ghost, on, the, highway, And, I,...","[you, be, a, ghost, on, the, highway, and, I, ...","[(PRON, PRP), (AUX, VBP), (DET, DT), (NOUN, NN...","[Ghost, Ghost]","[PERSON, PERSON]","[(Ghost), (Ghost)]"


In [1034]:
final_lyric_df.to_csv('MazzyStar_lyrics_with_spaCy_tags.csv')